In [1242]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import string

In [1303]:
all_reviews_dataframe = pd.read_csv(r'E:\parsing project\all_reviews_dataframe.csv')

with open(r'E:\parsing project\master_reviews_backup_batch_1', 'rb') as f:
  master_reviews_backup_batch1 = pickle.load(f)
  f.close()

batch1 = pd.DataFrame(master_reviews_backup_batch1['batch_1'].keys(), columns = ['author_date']).merge(pd.DataFrame(master_reviews_backup_batch1['batch_1'].values(), columns = ['review_text', 'product', 'rating out of 5']), left_index=True, right_index=True, how = 'left')

all_reviews_dataframe = pd.concat([all_reviews_dataframe, batch1], ignore_index= True)
perfumes_data = pd.read_csv(r'E:\parsing project\raw_data.csv')

In [1304]:
perfumes_data['index'] = perfumes_data['index'].str.rstrip(string.digits)
perfumes_data['index'] = perfumes_data['index'].str.lower()
perfumes_data = perfumes_data.drop_duplicates()
perfume_names = set(perfumes_data['index'].values)

In [1305]:
#lowercase all
all_reviews_dataframe['author_date'] = all_reviews_dataframe['author_date'].apply(lambda x: x.lower())
all_reviews_dataframe['review_text'] = all_reviews_dataframe['review_text'].apply(lambda x: x.lower())
all_reviews_dataframe['product'] = all_reviews_dataframe['product'].apply(lambda x: x.lower())
all_reviews_dataframe['rating out of 5'] = all_reviews_dataframe['rating out of 5'].apply(lambda x: x.lower())

In [1306]:
#drop these - no values
bad_ids = all_reviews_dataframe[(~all_reviews_dataframe['author_date'].str.contains('written')) & (all_reviews_dataframe['review_text'].str.contains('<'))].index
all_reviews_dataframe = all_reviews_dataframe.drop(labels=bad_ids)


In [1307]:
#remove dupes
all_reviews_dataframe = all_reviews_dataframe.drop_duplicates(subset = ['review_text', 'product', 'rating out of 5'])

# broken author-date

In [1308]:
all_reviews_dataframe['broken_author_date'] = np.where(~all_reviews_dataframe['author_date'].str.contains('written by'), 1, 0)

In [1309]:
#drop these - no values
bad_ad_id = all_reviews_dataframe[(all_reviews_dataframe['broken_author_date'] == 1) & (~all_reviews_dataframe['author_date'].str.contains('was this review helpful?'))].index
all_reviews_dataframe = all_reviews_dataframe.drop(labels = bad_ad_id)

In [1310]:
#review text has AD - swap with AD
review_written_by_id = all_reviews_dataframe[(all_reviews_dataframe['broken_author_date' ]== 1) & all_reviews_dataframe['author_date'].str.contains('was this review helpful?')]['review_text'].str.contains('written by').index
all_reviews_dataframe.loc[review_written_by_id,['author_date', 'review_text']] = all_reviews_dataframe.loc[review_written_by_id,['review_text', 'author_date']].values

In [1311]:
#product text has AD - swap with AD
product_written_by_id = all_reviews_dataframe.loc[(all_reviews_dataframe['product'].str.contains('written by')) & (all_reviews_dataframe['broken_author_date' ]== 1)].index
all_reviews_dataframe.loc[product_written_by_id,['author_date', 'product']] = all_reviews_dataframe.loc[product_written_by_id,['product', 'author_date']].values


In [1312]:
#rating has AD - swap with AD
ratings_written_by_id = all_reviews_dataframe.loc[(all_reviews_dataframe['rating out of 5'].str.contains('written by')) & (all_reviews_dataframe['broken_author_date' ]== 1)].index
all_reviews_dataframe.loc[ratings_written_by_id,['author_date', 'rating out of 5']] = all_reviews_dataframe.loc[ratings_written_by_id,['rating out of 5', 'author_date']].values

In [1313]:
def rat_split(row):
    try:
        j = int(len(row.split()))
    except:
        j = np.nan
    return j


In [1314]:
#all_reviews_dataframe['len_prod'] = all_reviews_dataframe['product'].apply(lambda x: len(x.split()))
#all_reviews_dataframe['len_rating'] = all_reviews_dataframe['rating out of 5'].apply(rat_split)
#all_reviews_dataframe['len_review_text'] = all_reviews_dataframe['review_text'].apply(lambda x: len(x.split()))

In [1315]:
#all_reviews_dataframe['len_prod_char'] = all_reviews_dataframe['product'].str.replace(' ', '').apply(lambda x: len(x))
#all_reviews_dataframe['len_rating_char'] = all_reviews_dataframe['rating out of 5'].str.replace(' ', '').apply(lambda x: len(x))
#all_reviews_dataframe['len_review_char'] = all_reviews_dataframe['review_text'].str.replace(' ', '').apply(lambda x: len(x))

In [1316]:
def len_x_char(df, excl = list()):
    for i in df.columns:
        if i not in excl:
            df[f'len_{i}'] = df[i].apply(lambda x: len(str(x).split()))
            df[f'len_char_{i}'] = df[i].str.replace(' ', '').apply(lambda x: len(str(x)))

In [1317]:
len_x_char(all_reviews_dataframe, excl = ['broken_author_date', 'author_date'])

# Fixing ratings

In [1318]:
#review_text and rating swap
review_text_rating_swap_ids = all_reviews_dataframe[(all_reviews_dataframe['len_review_text'] == 2) & (all_reviews_dataframe['review_text'].str.contains('star'))].index
all_reviews_dataframe.loc[review_text_rating_swap_ids,['review_text', 'rating out of 5']] = all_reviews_dataframe.loc[review_text_rating_swap_ids,['rating out of 5', 'review_text']].values

In [1319]:
len_x_char(all_reviews_dataframe, excl = ['broken_author_date', 'author_date', 'len_review_text', 'len_char_review_text', 'len_product', 'len_char_product', 'len_rating out of 5', 'len_char_rating out of 5'])

In [1320]:
#product and rating swap
product_rating_swap_ids = all_reviews_dataframe[(all_reviews_dataframe['len_product'] == 2) & (all_reviews_dataframe['product'].str.contains('star'))].index
all_reviews_dataframe.loc[product_rating_swap_ids,['product', 'rating out of 5']] = all_reviews_dataframe.loc[product_rating_swap_ids,['rating out of 5', 'product']].values

In [1321]:
len_x_char(all_reviews_dataframe, excl = ['broken_author_date', 'author_date', 'len_review_text', 'len_char_review_text', 'len_product', 'len_char_product', 'len_rating out of 5', 'len_char_rating out of 5'])

In [1322]:
for_nlp = all_reviews_dataframe[all_reviews_dataframe['review_text'].str.contains('was this review helpful?')]
for_nlp_id = all_reviews_dataframe[all_reviews_dataframe['review_text'].str.contains('was this review helpful?')].index

In [1323]:
all_reviews_dataframe

,author_date,review_text,product,rating out of 5,broken_author_date,len_review_text,len_char_review_text,len_product,len_char_product,len_rating out of 5,len_char_rating out of 5
0,"20300_written by jan on december 06, 2022",one of my favorites!!! i don't care when i wea...,kate spade live colorfullywomen,5 stars,0,22,85,4,28,2,6
1,"20301_written by ashley allure on june 17, 2022",i love it! i pondered on it for awhile. it’s e...,kate spade live colorfullywomen,5 stars,0,23,85,4,28,2,6
2,"20302_written by love it on february 19, 2022",love this and the travel spray bottle is so co...,kate spade live colorfullywomen,5 stars,0,15,60,4,28,2,6
3,"20303_written by cheryl on september 12, 2021",i love the fragrance which is light and not ov...,kate spade live colorfullywomen,5 stars,0,16,74,4,28,2,6
4,"20304_written by deirdre w. on july 16, 2021",pretty fragrance i get a lot of compliments on...,kate spade live colorfullywomen,5 stars,0,17,75,4,28,2,6
...,...,...,...,...,...,...,...,...,...,...,...
53493,"80324_written by dognutlv on november 18, 2021",hermes d'orange vert unisex,this is my favorite scent on my husband. it's ...,5 stars,0,4,24,35,156,2,6
53496,"80327_written by patty b on july 07, 2022",really nice fresh smell,kenneth cole blackwomen,4 stars,0,4,20,3,21,2,6
53497,"80328_written by stacey on march 21, 2022",i love the soft smell and the fact that it doe...,blue grasswomen,5 stars,0,18,67,2,14,2,6
53498,"80329_written by sylvia on april 10, 2022","omg!!! smells so good. it’s a sweet scent, not...",jean paul gaultier scandalwomen,5 stars,0,59,234,4,28,2,6


In [1324]:
# temporary drop - now all ratings are in-place
all_reviews_dataframe = all_reviews_dataframe.drop(labels=for_nlp_id)

In [1325]:
perfumes_data['name_char'] = perfumes_data['index'].apply(lambda x: len(str(x)))

In [1326]:
perfumes_data[perfumes_data['name_char'] == perfumes_data['name_char'].max()] #max length of perfume name - 57 chars, optimal length is 50 after manual inspection

,index,Price,nm,Design House,Fragrance Notes,Year,name_char
1099,lacoste eau de lacoste l.12.12 pour elle eau f...,$58.99,eau de toilette spray 3 oz,Lacoste,"Grapefruit, Yellow Mandarin, Indonesian Patcho...",2018,57


# Reviews and products fix

In [1327]:
# swap reviews and products if product name is greater than the longest perfume name
review_product_swap_by_max_name_id = all_reviews_dataframe[all_reviews_dataframe['len_char_product'] >= 50].index
all_reviews_dataframe.loc[review_product_swap_by_max_name_id,['review_text', 'product']] = all_reviews_dataframe.loc[review_product_swap_by_max_name_id,['product', 'review_text']].values

In [1328]:
len_x_char(all_reviews_dataframe, excl = ['broken_author_date', 'author_date', 'len_review_text', 'len_char_review_text', 'len_product', 'len_char_product', 'len_rating out of 5', 'len_char_rating out of 5'])

In [1329]:
perfumes = all_reviews_dataframe.loc[review_product_swap_by_max_name_id]['product'].unique()

In [1330]:
str_perfumes = '|'.join(perfumes)
str_perfumes = str_perfumes.replace(' ', '')
str_perfumes

"katespadelivecolorfullywomen|beyonceheatrushwomen|livejenniferlopezwomen|tommybahamasetsailstbartswomen|unforgivablewomanwomen|jovanislandgardeniawomen|michaelkorsroseradiantgoldwomen|voluptewomen|arianagrandethankunextwomen|bondno.9governorsislandunisex|modernmusewomen|orienticaamberrougeunisex|joywomen|parfumsdemarlylayton|valentinovocevivawomen|l'interditwomen|nishanehacivat|quelquesfleurswomen|emeraudewomen|cerruti1881women|tomfordtobaccovanille|obsessionnightperfume|gucciguiltyintensewomen|noawomen|eliesaabgirlofnowwomen|purewomen|reddoorwomen|usherwomen|pinksugarberryblastwomen|aliengoddessperfume|cleanwarmcottonwomen|cabotinewomen|irresistiblegivenchywomen|peacelove&juicycouturewomen|beyonceheatwomen|vanillafieldswomen|jessicamcclintock#3women|dknybedeliciousfreshblossomwomen|mancerarosesgreedy|laliquesoleilwomen|cabotinewomen|blackopiumintensewomen|shiseidozenwomen|couturecouturebyjuicycouturewomen|gucciflorawomen|angelnovawomen|bvlgariomniacoralwomen|ciara80%women|eaudusoirwo

In [1331]:
perfumes_from_df = perfumes_data['index'].unique()
str_perfumes_df = '|'.join(perfumes_from_df)
str_perfumes_df = str_perfumes_df.replace(' ', '')

In [1332]:
#all reviews that have matching strings in the review_text
review_product_swap_fullmatch_id = all_reviews_dataframe[all_reviews_dataframe['review_text'].apply(lambda x: x.replace(' ', '')).str.fullmatch(str_perfumes)].index
all_reviews_dataframe.loc[review_product_swap_fullmatch_id,['review_text', 'product']] = all_reviews_dataframe.loc[review_product_swap_fullmatch_id,['product', 'review_text']].values

In [1333]:
len_x_char(all_reviews_dataframe, excl = ['broken_author_date', 'author_date', 'len_review_text', 'len_char_review_text', 'len_product', 'len_char_product', 'len_rating out of 5', 'len_char_rating out of 5'])

In [1334]:
#all reviews that have matching strings in the review_text 2
review_product_swap_fullmatch_id2 = all_reviews_dataframe[all_reviews_dataframe['review_text'].apply(lambda x: x.replace(' ', '')).str.fullmatch(str_perfumes_df)].index
all_reviews_dataframe.loc[review_product_swap_fullmatch_id2,['review_text', 'product']] = all_reviews_dataframe.loc[review_product_swap_fullmatch_id2,['product', 'review_text']].values

In [1335]:
len_x_char(all_reviews_dataframe, excl = ['broken_author_date', 'author_date', 'len_review_text', 'len_char_review_text', 'len_product', 'len_char_product', 'len_rating out of 5', 'len_char_rating out of 5'])

In [1336]:
pd.set_option('display.max_rows', 30)

In [1337]:
#len_x_char(all_reviews_dataframe, excl = ['broken_author_date', 'author_date', 'len_review_text', 'len_char_review_text', 'len_product', 'len_char_product', 'len_rating out of 5', 'len_char_rating out of 5'])

In [1338]:
replace_ids = [28561, 28654, 10032, 25496, 26965, 11420, 26468, 27710, 27643, 25057,  52611, 52623, 28427, 25924, 24759, 24521, 28103, 28104] #manual inspection
all_reviews_dataframe.loc[replace_ids,['review_text', 'product']] = all_reviews_dataframe.loc[replace_ids,['product', 'review_text']].values


In [1339]:
len_x_char(all_reviews_dataframe, excl = ['broken_author_date', 'author_date', 'len_review_text', 'len_char_review_text', 'len_product', 'len_char_product', 'len_rating out of 5', 'len_char_rating out of 5'])

In [1340]:
replace_id_rating = [27743, 27744, 27745]
all_reviews_dataframe.loc[replace_id_rating,['product', 'rating out of 5']] = all_reviews_dataframe.loc[replace_id_rating,['rating out of 5', 'product']].values

In [1404]:
all_reviews_dataframe[all_reviews_dataframe['author_date'].str.contains('love this fragrance')]

,author_date,review_text,product,rating out of 5,broken_author_date,len_review_text,len_char_review_text,len_product,len_char_product,len_rating out of 5,len_char_rating out of 5
26086,46386_written by your gonna love this fragranc...,"this fragrance is amazing, bought it for my si...",sean john empresswomen,5 stars,0,27,122,3,20,2,6
27546,47846_written by love this fragrance on septem...,everything from this line are true to the name...,roger & gallet verveine utopie,5 stars,0,37,191,5,26,2,6


In [1341]:
len_x_char(all_reviews_dataframe, excl = ['broken_author_date', 'author_date', 'len_review_text', 'len_char_review_text', 'len_product', 'len_char_product', 'len_rating out of 5', 'len_char_rating out of 5'])

# for nlp part

In [1342]:
for_nlp['review_text'] = np.nan

C:\Users\Rustem K\AppData\Local\Temp\ipykernel_17624\2935360098.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for_nlp['review_text'] = np.nan


In [1343]:
product_rating_swap_for_nlp = for_nlp[for_nlp['product'].apply(lambda x: x.replace(' ', '')).str.fullmatch(str_perfumes)].index
for_nlp.loc[product_rating_swap_for_nlp,['review_text', 'rating out of 5']] = for_nlp.loc[product_rating_swap_for_nlp,['rating out of 5', 'review_text']].values

C:\Users\Rustem K\AppData\Local\Temp\ipykernel_17624\3654601660.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for_nlp.loc[product_rating_swap_for_nlp,['review_text', 'rating out of 5']] = for_nlp.loc[product_rating_swap_for_nlp,['rating out of 5', 'review_text']].values


In [1344]:
swap_for_nlp = for_nlp[for_nlp['review_text'].isna()].index
for_nlp[for_nlp['review_text'].isna()]
for_nlp.loc[swap_for_nlp,['review_text', 'product', 'rating out of 5']] = for_nlp.loc[swap_for_nlp,['product', 'rating out of 5', 'review_text']].values

In [1345]:
match_this = "it has been one of my all time favorites for years |marry me! by lanvin is a very flowery| i have worn manifesto | is a very flowery | my absolute favorite perfume! | love this fragrance. | salvador dali is an exquisite |lizsport is my favorite go-anywhere fragrance | patou 1000: black violets and rose kennedy !! timeless| 1000- i cant say enough about this perfume | is so seductive| i just got it! | this is so cute | i first purchased this fragrance| i love this smell, it | i absolutely love this fragrance, and i | perry ellis love -- wears very well.| this perfume has a light flowery | agua de loewe ella been my favorite perfume| it smells like honey. | cartier bleu....s!ella so good!!!!!!!| being a first timer | purple light--this fragrance is awesome!!!!| i get lots of compliments on miller harris | marc jacobs daisy blush is a beautiful scent".replace(' ', '')
match_this

#for_nlp['product'].unique()

'ithasbeenoneofmyalltimefavoritesforyears|marryme!bylanvinisaveryflowery|ihavewornmanifesto|isaveryflowery|myabsolutefavoriteperfume!|lovethisfragrance.|salvadordaliisanexquisite|lizsportismyfavoritego-anywherefragrance|patou1000:blackvioletsandrosekennedy!!timeless|1000-icantsayenoughaboutthisperfume|issoseductive|ijustgotit!|thisissocute|ifirstpurchasedthisfragrance|ilovethissmell,it|iabsolutelylovethisfragrance,andi|perryellislove--wearsverywell.|thisperfumehasalightflowery|aguadeloeweellabeenmyfavoriteperfume|itsmellslikehoney.|cartierbleu....s!ellasogood!!!!!!!|beingafirsttimer|purplelight--thisfragranceisawesome!!!!|igetlotsofcomplimentsonmillerharris|marcjacobsdaisyblushisabeautifulscent'

In [1346]:
replace_these_ids = for_nlp[for_nlp['product'].apply(lambda x: x.replace(' ', '')).str.match(match_this)].index
replace_these_ids = np.append(replace_these_ids, 24496)

for_nlp.loc[replace_these_ids,['review_text', 'product']] = for_nlp.loc[replace_these_ids,['product','review_text']].values

In [1347]:
len_x_char(for_nlp, excl = ['broken_author_date', 'author_date', 'len_review_text', 'len_char_review_text', 'len_product', 'len_char_product', 'len_rating out of 5', 'len_char_rating out of 5'])

C:\Users\Rustem K\AppData\Local\Temp\ipykernel_17624\2528629390.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'len_{i}'] = df[i].apply(lambda x: len(str(x).split()))
C:\Users\Rustem K\AppData\Local\Temp\ipykernel_17624\2528629390.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'len_char_{i}'] = df[i].str.replace(' ', '').apply(lambda x: len(str(x)))


In [1348]:
#add nlp part back
reviews_fixed = pd.concat([all_reviews_dataframe, for_nlp], ignore_index= True)

In [1349]:
reviews_fixed = reviews_fixed.drop(['broken_author_date', 'len_review_text', 'len_char_review_text', 'len_product', 'len_char_product', 'len_rating out of 5', 'len_char_rating out of 5'], axis = 1) 

In [1720]:
reviews_fixed[reviews_fixed['product'].str.contains('star')]

,author_date,review_text,product,rating out of 5,date,authors
2544,"23161_written by anonymous on september 11, 2022",great fragrance. very unisex. it lingers aroun...,montale paris starry nights,5 stars,2022-09-11,ymous _2544
2545,"23162_written by capturedbyfire on april 08, 2022",just received my bottle of montale paris starr...,montale paris starry nights,5 stars,2022-04-08,fire
2546,"23163_written by johnl on december 10, 2021",bottle feels like it is not full .other than t...,montale paris starry nights,5 stars,2021-12-10,johnl
2547,23164_written by passionate4perfume on novembe...,starry nights a wonderful example of truly uni...,montale paris starry nights,5 stars,2020-11-23,ate4perfume
2753,"23387_written by erin on june 23, 2022",i’m so excited this is available. i wore this ...,sun moon stars,5 stars,2022-06-23,erin
2754,"23389_written by frenchghost on april 07, 2022",i bought this out of curiosity&beautiful bottl...,sun moon stars,5 stars,2022-04-07,frenchghost
2755,"23390_written by dale e on february 17, 2022","crisp, floral and clean! a beautiful perfume w...",sun moon stars,5 stars,2022-02-17,dale e
2756,"23391_written by anonymous on october 03, 2021",pleasantly surprised to see sun moon and stars...,sun moon stars,4 stars,2021-10-03,ymous _2756
18476,"44760_written by lynn on december 16, 2020",absolutely love this fragrance!! it’s a mild f...,michael kors starlight shimmer,5 stars,2020-12-16,lynn
18773,"45108_written by amy on november 23, 2021",this scent is beautiful very unique. it is lig...,moschino cheap & chic stars,4 stars,2021-11-23,amy


In [1372]:
reviews_fixed = reviews_fixed.drop_duplicates(subset = ['review_text', 'rating out of 5', 'product'])


,author_date,review_text,product,rating out of 5
0,"20300_written by jan on december 06, 2022",one of my favorites!!! i don't care when i wea...,kate spade live colorfullywomen,5 stars
1,"20301_written by ashley allure on june 17, 2022",i love it! i pondered on it for awhile. it’s e...,kate spade live colorfullywomen,5 stars
2,"20302_written by love it on february 19, 2022",love this and the travel spray bottle is so co...,kate spade live colorfullywomen,5 stars
3,"20303_written by cheryl on september 12, 2021",i love the fragrance which is light and not ov...,kate spade live colorfullywomen,5 stars
4,"20304_written by deirdre w. on july 16, 2021",pretty fragrance i get a lot of compliments on...,kate spade live colorfullywomen,5 stars
...,...,...,...,...
46049,"written by seminole sweetie on september 15, 2016","i wear this fragrance, especially in the sprin...",burberry weekend for women,NaN
46050,"written by sm on march 29, 2016",i love love love love love this perfume. when ...,burberry weekend for women,NaN
46051,"written by alice on march 03, 2016",burbbery weekend is a wonderful strong scent w...,burberry weekend for women,NaN
46052,"written by kristal on december 17, 2015",this fragrance smell so good it's such a matur...,burberry weekend for women,NaN


In [1411]:
reviews_fixed.loc[43920, ['author_date', 'review_text']] = reviews_fixed.loc[43920, ['review_text', 'author_date']].values


C:\Users\Rustem K\AppData\Local\Temp\ipykernel_17624\4093158475.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_fixed.loc[43920, ['author_date', 'review_text']] = reviews_fixed.loc[43920, ['review_text', 'author_date']].values


In [1450]:
import re
from dateutil.parser import parse

delimiters = "by", "on"
regex_pattern = '|'.join(map(re.escape, delimiters))

In [1452]:
reviews_fixed['date'] = reviews_fixed['author_date'].apply(lambda x: re.split(regex_pattern, x)[-1])

C:\Users\Rustem K\AppData\Local\Temp\ipykernel_17624\3973846707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_fixed['date'] = reviews_fixed['author_date'].apply(lambda x: re.split(regex_pattern, x)[-1])


In [1454]:
reviews_fixed.loc[43920, ['date']] = 'march 31, 2015'

In [1455]:
reviews_fixed.loc[43920]

author_date                   written by anonymous on march 31, 2015
review_text        absolutely love this fragrance. it has a fresh...
product                                        eau de cartier unisex
rating out of 5                                                  NaN
date                                                  march 31, 2015
authors                                                       ymous 
Name: 43920, dtype: object

In [1456]:
reviews_fixed['authors'] = reviews_fixed['author_date'].apply(lambda x: re.split(regex_pattern, x)[-2])

C:\Users\Rustem K\AppData\Local\Temp\ipykernel_17624\2564483291.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_fixed['authors'] = reviews_fixed['author_date'].apply(lambda x: re.split(regex_pattern, x)[-2])


In [1457]:
reviews_fixed['date'] = reviews_fixed['date'].apply(lambda x: parse(x))

C:\Users\Rustem K\AppData\Local\Temp\ipykernel_17624\3468182198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_fixed['date'] = reviews_fixed['date'].apply(lambda x: parse(x))


In [1458]:
reviews_fixed

,author_date,review_text,product,rating out of 5,date,authors
0,"20300_written by jan on december 06, 2022",one of my favorites!!! i don't care when i wea...,kate spade live colorfullywomen,5 stars,2022-12-06,jan
1,"20301_written by ashley allure on june 17, 2022",i love it! i pondered on it for awhile. it’s e...,kate spade live colorfullywomen,5 stars,2022-06-17,ashley allure
2,"20302_written by love it on february 19, 2022",love this and the travel spray bottle is so co...,kate spade live colorfullywomen,5 stars,2022-02-19,love it
3,"20303_written by cheryl on september 12, 2021",i love the fragrance which is light and not ov...,kate spade live colorfullywomen,5 stars,2021-09-12,cheryl
4,"20304_written by deirdre w. on july 16, 2021",pretty fragrance i get a lot of compliments on...,kate spade live colorfullywomen,5 stars,2021-07-16,deirdre w.
...,...,...,...,...,...,...
46049,"written by seminole sweetie on september 15, 2016","i wear this fragrance, especially in the sprin...",burberry weekend for women,NaN,2016-09-15,seminole sweetie
46050,"written by sm on march 29, 2016",i love love love love love this perfume. when ...,burberry weekend for women,NaN,2016-03-29,sm
46051,"written by alice on march 03, 2016",burbbery weekend is a wonderful strong scent w...,burberry weekend for women,NaN,2016-03-03,alice
46052,"written by kristal on december 17, 2015",this fragrance smell so good it's such a matur...,burberry weekend for women,NaN,2015-12-17,kristal


In [1658]:
reviews_fixed[reviews_fixed.duplicated(subset = ['review_text', 'date', 'authors'])]

,author_date,review_text,product,rating out of 5,date,authors,womenuniflag
2385,"22972_written by sue on february 29, 2020",quatre cologne was purchased based on reviews ...,boucheron quatre,3 stars,2020-02-29,sue,boucheron quatrewomen
4631,"25472_written by ms. taylor on december 21, 2021",i actually like it! it smells very chic and cl...,kenneth cole white,5 stars,2021-12-21,ms. taylor,kenneth cole whitewomen
5480,"26410_written by flowerpower girl on april 15,...",loved it! light and airy. perfect for spring o...,miss dior blooming bouquet set,5 stars,2022-04-15,flowerpower girl,miss dior blooming bouquet set
8135,"29388_written by gen j on may 18, 2020",this was a gift for my aunt and she loves it.,michael kors gold luxe edition,5 stars,2020-05-18,gen j,michael kors gold luxe editionwomen
10037,"31538_written by evelyn on december 23, 2019",i love all vince camuto fragrance but this one...,vince camuto capri,5 stars,2019-12-23,evelyn,vince camuto capriwomen
...,...,...,...,...,...,...,...
43045,"80292_written by sherry juarez on november 04,...",love love this soft floral woodsy italian scen...,valentino voce viva,5 stars,2022-11-04,sherry juarez,valentino voce viva
43049,"80296_written by bluenerd on january 19, 2023","smells fresh, a compliment getter and good for...",afnan 9 pm,5 stars,2023-01-19,bluenerd,afnan 9 pm
43066,"80316_written by anonymous on september 11, 2022",great fragrance. very unisex. it lingers aroun...,montale paris starry nights,5 stars,2022-09-11,ymous,montale paris starry nights unisex
43074,"80324_written by dognutlv on november 18, 2021",this is my favorite scent on my husband. it's ...,hermes d'orange vert,5 stars,2021-11-18,dognutlv,hermes d'orange vert unisex


In [1659]:
reviews_fixed = reviews_fixed.drop_duplicates(subset = ['review_text', 'date', 'authors'])

In [1696]:
reviews_fixed.loc[reviews_fixed['authors'].str.contains('ymous'), 'authors']= reviews_fixed.loc[reviews_fixed['authors'].str.contains('ymous'), 'authors']+ '_' + reviews_fixed.loc[reviews_fixed['authors'].str.contains('ymous')].index.astype(str).values

In [1697]:
reviews_fixed[reviews_fixed['authors'].str.contains('ymous')]

,author_date,review_text,product,rating out of 5,date,authors,womenuniflag
6,"20306_written by anonymous on november 03, 2020",i bought this fragrance because i tried in the...,kate spade live colorfully,5 stars,2020-11-03,ymous _6,kate spade live colorfullywomen
8,"20308_written by anonymous on may 15, 2020",i love it! i will definitely be purchasing a b...,kate spade live colorfully,4 stars,2020-05-15,ymous _8,kate spade live colorfullywomen
10,"20312_written by anonymous on february 11, 2020",i love live colorfully people like it alot whe...,kate spade live colorfully,5 stars,2020-02-11,ymous _10,kate spade live colorfullywomen
11,"20313_written by anonymous on february 09, 2020",kate spade live colorfully is the best fragran...,kate spade live colorfully,5 stars,2020-02-09,ymous _11,kate spade live colorfullywomen
17,"20319_written by anonymous on may 13, 2018",i love this perfume. just right for me.,kate spade live colorfully,5 stars,2018-05-13,ymous _17,kate spade live colorfullywomen
...,...,...,...,...,...,...,...
46041,"written by anonymous on january 15, 2015",i love this fragrance.....i wear it to work an...,ed hardy,NaN,2015-01-15,ymous _46041,ed hardywomen
46044,"written by anonymous on december 30, 2017",the burberry weekend came without a top the bo...,burberry weekend,NaN,2017-12-30,ymous _46044,burberry weekend
46045,"written by anonymous on september 08, 2017",wonderful fragrance! it blends with your body ...,burberry weekend,NaN,2017-09-08,ymous _46045,burberry weekend
46047,"written by anonymous on february 06, 2017",its not long lasting,burberry weekend,NaN,2017-02-06,ymous _46047,burberry weekend


In [1520]:
reviews_fixed['product'] = np.where(reviews_fixed['product'].apply(lambda x: x.replace(' ','')[-8:]).str.contains('forwomen'),
                                         reviews_fixed['product'].apply(lambda x: ' '.join(x.split(' ')[:-2])), reviews_fixed['product'])

C:\Users\Rustem K\AppData\Local\Temp\ipykernel_17624\3377038212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_fixed['product'] = np.where(reviews_fixed['product'].apply(lambda x: x.replace(' ','')[-8:]).str.contains('forwomen'),


In [1552]:
reviews_fixed['product'] = np.where(reviews_fixed['product'].apply(lambda x: x.split(' ')[-1]).str.contains('unisex'),
                                         reviews_fixed['product'].apply(lambda x: ' '.join(x.split(' ')[:-1])), reviews_fixed['product'])

C:\Users\Rustem K\AppData\Local\Temp\ipykernel_17624\3586360648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_fixed['product'] = np.where(reviews_fixed['product'].apply(lambda x: x.split(' ')[-1]).str.contains('unisex'),


In [1550]:
reviews_fixed['product'] = np.where(reviews_fixed['product'].apply(lambda x: x.replace(' ','')[-5:]).str.contains('women'),
                                         reviews_fixed['product'].apply(lambda x: ' '.join(x.split(' ')[:-1]) + ' ' + x.split(' ')[-1][:-5]).str.strip(), reviews_fixed['product'])

C:\Users\Rustem K\AppData\Local\Temp\ipykernel_17624\1603785837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_fixed['product'] = np.where(reviews_fixed['product'].apply(lambda x: x.replace(' ','')[-5:]).str.contains('women'),


In [1555]:
reviews_fixed.to_csv('reviews_clean.csv', index = False)

2757

In [1557]:
perfumes_data['index'] = np.where(perfumes_data['index'].apply(lambda x: x.replace(' ','')[-8:]).str.contains('forwomen'),
                                  perfumes_data['index'].apply(lambda x: ' '.join(x.split(' ')[:-2])), perfumes_data['index'])

In [1558]:
perfumes_data['index']  = np.where(perfumes_data['index'] .apply(lambda x: x.split(' ')[-1]).str.contains('unisex'),
                                   perfumes_data['index'] .apply(lambda x: ' '.join(x.split(' ')[:-1])), perfumes_data['index'])

In [1560]:
perfumes_data['index'] = np.where(perfumes_data['index'].apply(lambda x: x.replace(' ','')[-5:]).str.contains('women'),
                                         perfumes_data['index'].apply(lambda x: ' '.join(x.split(' ')[:-1]) + ' ' + x.split(' ')[-1][:-5]).str.strip(), perfumes_data['index'])

In [1561]:
len(set(perfumes_data['index']))

2725

In [1702]:
reviews_fixed.to_csv('reviews_clean.csv', index = False)

# Check

In [1719]:
#sanity check - we are missing 59 entires - some are not perfumes
not_matched = [i for i in reviews_fixed['product'].unique() if i not in perfumes_data['index'].unique()]
len(set(not_matched))

12

In [1582]:
not_matched

['cerruti 1881',
 'mancera roses greedy',
 'escentric molecule 02',
 'ck2',
 'flowerbomb nectar',
 'narciso rodriguez pure musc',
 'boucheron quatre',
 'montale paris starry nights',
 'elie saab girl of now forever',
 'believe britney spears',
 'nishane ani',
 'meow',
 'daisy love eau so sweet',
 'juicy couture i am juicy couture',
 'versace yellow diamond intense',
 'm by mariah carey',
 'm by mariah carey luscious pink',
 'symétrie bliss',
 'ck one deodorant',
 'obsession perfume set',
 'escentric 02',
 'curve chill',
 'burberry body tender',
 'alyssa ashley musk lotion',
 '273 indigo',
 'ck one spray',
 'rose the one',
 'fred hayman 273',
 'cool water lotion',
 'paris hilton tease',
 'chanel no5',
 'white tea body cream',
 'bond no. 9 b-9',
 'escentric 04',
 'zarkoperfume pink molecule 090.09',
 'bois 1920',
 'franck olivier sun java',
 'dancing by jessica mcclintock',
 'delicious strawberry',
 'extraordinary petale',
 'essence de bois',
 'dora the explorer',
 'fcuk luscious vanilla

In [1644]:
#find the missing entries on the web and append them manually

missings = [
    
    {'index': 'cerruti 1881', 'Price': '$50.99', 'nm': 'eau de toilette spray 3.4 oz', 'Design House': 'Nino Cerruti', 'Fragrance Notes': 'Jasmine, Freesia, Rose, Violet, Bergamot, Iris, Lily of the Valley, Mimosa, Galbanum, Narcissus, Chamomile, Geranium, Iris, Orange Blossom, Tuberose, Brazilian Rosewood, Coriander, Jasmine, Cedar, Vanilla, Musk, Amber, Sandalwood', 'Year': '1995'},
    
    {'index': 'mancera roses greedy', 'Price': '$124.99', 'nm': 'eau de parfum spray 4 oz', 'Design House': 'Mancera', 'Fragrance Notes': 'Black Currant, Peach, White Musk, Sugar, Vanilla', 'Year': '2012'},
    
    {'index': 'escentric molecule 02', 'Price': '$129.99', 'nm': 'eau de toilette spray 3.5 oz', 'Design House': 'Escentric Molecules', 'Fragrance Notes': 'Ambroxan', 'Year': '2008'},
    
    {'index': 'ck2', 'Price': '$15.99', 'nm': 'eau de toilette spray 0.68 oz', 'Design House': 'Calvin Klein', 'Fragrance Notes': 'Mandarin Orange, Wasabi, Hedione, Rose, Sandalwood', 'Year': '2016'},
    
    {'index': 'flowerbomb nectar', 'Price': '$204.99', 'nm': 'eau de parfum intense spray 3 oz', 'Design House': 'Viktor & Rolf', 'Fragrance Notes': 'Tonka Bean, Vanilla, Patchouli, Osmanthus, Orange Blossom', 'Year': '2018'},
    
    {'index': 'narciso rodriguez pure musc', 'Price': '$125.99', 'nm': 'eau de parfum spray 3.3 oz', 'Design House': 'Narciso Rodriguez', 'Fragrance Notes': 'Musk, Flowers, Cashmeran', 'Year': '2019'},
    
    {'index': 'boucheron quatre', 'Price': '$61.99', 'nm': 'eau de parfum spray 3.3 oz', 'Design House': 'Boucheron', 'Fragrance Notes': 'Lemon, Lime, Violet Leaf, Jasmine Sambac, Rose', 'Year': '2015'},
    
    {'index': 'montale paris starry nights', 'Price': '$129.99', 'nm': 'eau de parfum spray 3.4 oz', 'Design House': 'Montale', 'Fragrance Notes': 'Rose, Jasmine, Bergamot, Apple, Lemon', 'Year': '2015'},
    
    {'index': 'elie saab girl of now forever', 'Price': '$58.99', 'nm': 'eau de parfum spray 1.6 oz', 'Design House': 'Elie Saab', 'Fragrance Notes': 'Rose, Orange Blossom, Vanilla, Patchouli, Almond', 'Year': '2019'},
    
    {'index': 'believe britney spears', 'Price': '$26.99', 'nm': 'Eau De Parfum Spray 3.3 oz', 'Design House': 'Britney Spears', 'Fragrance Notes': 'Guava, Tangerine, Honeysuckle, Lime blossom, Praline', 'Year': '2007'},
    
    {'index': 'nishane ani', 'Price': '$206.99', 'nm': 'Extrait De Parfum Spray 1.7 oz', 'Design House': 'Nishane', 'Fragrance Notes': 'Ginger, Bergamot, Pink Pepper, Turkish Rose, Vanilla', 'Year': '2019'},
    
    {'index': 'meow', 'Price': '$45.99', 'nm': 'Eau De Parfum Spray 3.4 oz', 'Design House': 'Katy Perry', 'Fragrance Notes': 'Jasmine, Gardenia, Tangerine, Pear, Honeysuckle', 'Year': '2011'},
    
    {'index': 'daisy love eau so sweet', 'Price': '$134.99', 'nm': 'Eau De Toilette Spray 3.3 oz', 'Design House': 'Marc Jacobs', 'Fragrance Notes': 'Raspberry, Bergamot, Blackberry, Daisy, Musk', 'Year': '2019'},
    
    {'index': 'juicy couture i am juicy couture', 'Price': '$64.99', 'nm': 'eau de parfum spray 3.4 oz', 'Design House': 'Juicy Couture', 'Fragrance Notes': 'Gardenia, Sweet Pea, Heliotrope, Rose, Musk', 'Year': '2015'},
    
    {'index': 'versace yellow diamond intense', 'Price': '$90.99', 'nm': 'eau de parfum spray (new packaging) 3 oz', 'Design House': 'Gianni Versace', 'Fragrance Notes': 'Neroli, Citron, Pear, Bergamot, Orange Blossom', 'Year': '2014'},

    {'index': 'm by mariah carey', 'Price': '$44.99', 'nm': 'm by mariah carey', 'Design House': 'Mariah Carey', 'Fragrance Notes': 'Marshmallow, Incense, Tiare flower, Gardenia, Patchouli', 'Year': '2007'},
    
    {'index': 'm by mariah carey luscious pink', 'Price': '$46.99', 'nm': 'Eau De Parfum Spray 3.3 oz', 'Design House': 'Mariah Carey', 'Fragrance Notes': 'Sicilian bergamot, Bellini, Sea notes, Tiare, Lily of the valley', 'Year': '2008'},
   
    {'index': 'symétrie bliss', 'Price': '$34.99', 'nm': 'Eau De Parfum Spray 3.4 oz', 'Design House': 'Symétrie', 'Fragrance Notes': 'White pear, freesia, rose, peonies, violet, patchouli, amber and vanilla absolute', 'Year': '2017'},
    
    {'index': 'escentric 02', 'Price': '$89.99', 'nm': 'eau de toilette spray 3.5 oz', 'Design House': 'Escentric Molecules', 'Fragrance Notes': 'jasmine, iris, ambroxan, iso E Super', 'Year': '2008'},
    
    {'index': 'curve chill', 'Price': '$21.99', 'nm': 'Eau De Toilette Spray 3.4 oz', 'Design House': 'Liz Claiborne', 'Fragrance Notes': 'Lemon, Green Tea, Ginger, Freesia, Peony', 'Year': '2006'},
    
    {'index': 'burberry body tender', 'Price': '$9.99', 'nm': 'eau de toilette mini 0.15 oz', 'Design House': 'Burberry', 'Fragrance Notes': 'Apple, Wormwood, Peach, Lemon, Rose', 'Year': '2013'},
    
    {'index': '273 indigo', 'Price': '$0.99', 'nm': 'eau de parfum vial 0.05 oz', 'Design House': 'Fred Hayman', 'Fragrance Notes': 'Orange, Peach, Jasmine, Carnation, Vanilla', 'Year': '2003'},
    
    {'index': 'rose the one', 'Price': '$5.99', 'nm': 'Eau De Parfum Spray Vial 0.05 oz', 'Design House': 'Dolce & Gabbana', 'Fragrance Notes': 'Madonna lily, pink grapefruit, blackcurrant, lily of the valley, sandalwood, litchi, peony, ambrette seed, musk, vanilla, mandarin, rose', 'Year': '2009'},


    {'index': 'fred hayman 273', 'Price': '$32.99', 'nm': 'Eau De Parfum Spray 2.5 oz', 'Design House': 'Fred Hayman', 'Fragrance Notes': 'gardenia, exotic fruits, amber, sandalwood', 'Year': '1989'},
    
    {'index': 'paris hilton tease', 'Price': '$29.99', 'nm': 'eau de parfum spray 3.4 oz', 'Design House': 'Paris Hilton', 'Fragrance Notes': 'Apples, Peaches, Bergamot, White Flowers, Amber, Hot Sand', 'Year': '2010'},
    
    {'index': 'chanel no5', 'Price': '$159.99', 'nm': 'Eau De Parfum 3.4 oz', 'Design House': 'Chanel', 'Fragrance Notes': 'Jasmine, Rose, Lily-of-the-valley, Vanilla, Sandalwood', 'Year': '1986'},
    
    {'index': 'bond no. 9 b-9', 'Price': '$389.99', 'nm': 'Eau De Parfum Spray 3.3 oz', 'Design House': 'Bond No. 9', 'Fragrance Notes': 'Musk, Amber, Jasmine, Orchid, Sandalwood', 'Year': '2015'},
    
    {'index': 'escentric 04', 'Price': '$73.99', 'nm': 'escentric 04', 'Design House': 'Escentric Molecules', 'Fragrance Notes': 'Sandalwood', 'Year': '2017'},
    
    {'index': 'zarkoperfume pink molecule 090.09', 'Price': '$155.99', 'nm': 'Eau De Parfum Spray 3.3 oz', 'Design House': 'Zarkoperfume', 'Fragrance Notes': 'Champagne, Black Orchid, Apricot, Black Elder, Blackwood, Whipped Cream, Mahogany', 'Year': '2014'},
    
    {'index': 'bois 1920', 'Price': '$113.99', 'nm': 'sushi imperiale eau de parfum spray 1.7 oz', 'Design House': 'Bois 1920', 'Fragrance Notes': 'Tobacco, Amber, Musk, Cedar, Apricot, Lavender', 'Year': '2005'},
    
    {'index': 'franck olivier sun java', 'Price': '$30.99', 'nm': 'eau de toilette spray 2.5 oz', 'Design House': 'Franck Olivier', 'Fragrance Notes': 'Mandarin Orange, Mint, Bergamot, Watermelon, Amber', 'Year': '2005'},
    
    {'index': 'dancing by jessica mcclintock', 'Price': '$40.99', 'nm': 'eau de parfum spray 3.4 oz', 'Design House': 'Jessica Mcclintock', 'Fragrance Notes': 'Bergamot, Lemon, Apple, Jasmine, Cedar', 'Year': '2013'},
    
    {'index': 'extraordinary petale', 'Price': '$28.99', 'nm': 'eau de parfum spray 3 oz', 'Design House': 'Oscar De La Renta', 'Fragrance Notes': 'Champagne, Bergamot, Jasmime, Rose,Musk', 'Year': '2016'},
    
    {'index': 'essence de bois', 'Price': '$99.99', 'nm': 'eau de parfum spray 3.4 oz', 'Design House': 'Cartier', 'Fragrance Notes': 'Bergamot, Yuzu, Violet leaf, Lavender, Violet', 'Year': '2012'},
    
    {'index': 'mon guerlain florale', 'Price': '$137.99', 'nm': 'eau de parfum spray 1.6 oz', 'Design House': 'Guerlain', 'Fragrance Notes': 'Jasmine Sambac, floral notes, Neroli, Peach, Vanilla', 'Year': '2018'},
    
    {'index': 'banana republic oud mosaic 17', 'Price': '$94.99', 'nm': 'eau de parfum spray 2.5 oz', 'Design House': 'Banana Republic', 'Fragrance Notes': 'White Pepper, Cardamom, Turkish Rose, Labdanum, Oud', 'Year': '2017'},
    
    {'index': 'emor london oud no. 5', 'Price': '$127.99', 'nm': 'eau de parfum spray 4.2 oz', 'Design House': 'Emor London', 'Fragrance Notes': 'Mint, Lemon, Bergamot, Lavender, White Flowers, Jasmine, Nutmeg, Agarwood, Cedar, Cardamom, Patchouli', 'Year': '2015'},

    {'index': 'eau de sisley 2', 'Price': '$171.99', 'nm': 'eau de toilette spray 3.3 oz', 'Design House': 'Sisley', 'Fragrance Notes': 'Bergamot, Basil, Cardamom, Jasmine, Iris', 'Year': '2009'},
    
    {'index': 'lalique noir premier fleur universelle 1900', 'Price': '$174.99', 'nm': 'eau de parfum spray 3.3 oz', 'Design House': 'Lalique', 'Fragrance Notes': 'immortelle, patchouli, lorenox, myrhh, cinnamon', 'Year': '2014'},
    
    {'index': 'ck one cologne', 'Price': '$52.99', 'nm': 'eau de toilette spray 6.7 oz', 'Design House': 'Calvin Klein', 'Fragrance Notes': 'Bergamot, Papaya, Lemon, Mandarin Orange, Cardamom, Green Notes, Pineapple, Lily of the Valley, Jasmine, Orris Root, Rose, Violet, Nutmeg, Oakmoss, Cedar, Musk, Amber, Sandalwood', 'Year': '1994'},
    
    {'index': 'baccarat rouge 540', 'Price': '$489.99', 'nm': 'eau de parfum spray 2.4 oz', 'Design House': 'Maison Francis', 'Fragrance Notes': 'Jasmine, Amberwood, Ambergris, Saffron, Cedar', 'Year': '2015'},
    
    {'index': 'elizabeth arden white tea', 'Price': '$42.99', 'nm': 'eau de toilette spray 3.3 oz', 'Design House': 'Elizabeth Arden', 'Fragrance Notes': 'Mandarin Orange, Clary Sage, Fern, Sea Notes, White Tea', 'Year': '2017'},
    
    {'index': 'guerlain santal royal', 'Price': '$263.99', 'nm': 'eau de parfum spray 4.2 oz', 'Design House': 'Guerlain Santal Royal', 'Fragrance Notes': 'Jasmine, Neroli, Rose, Cinnamon, Leather', 'Year': '2016'},
    
    {'index': 'obsession perfume', 'Price': '$42.99', 'nm': 'eau de parfum spray 3.4 oz', 'Design House': 'Calvin Klein', 'Fragrance Notes': 'Basil, Peach, Vanilla, Lemon, Bergamot, Mandarin Orange, Green Notes, Jasmine, Orange Blossom, Rose, Cedar, Sandalwood, Coriander, Oakmoss, Spices, Vetiver, Vanilla, Incense, Civet, Musk, Amber', 'Year': '1985'},
    
    {'index': 'moschino toy 2', 'Price': '$72.99', 'nm': 'eau de parfum spray 3.4 oz', 'Design House': 'Moschino', 'Fragrance Notes': 'Peony, Jasmine, Amberwood, Musk, Apple', 'Year': '2018'},
    
    {'index': 'my burberry black perfume', 'Price': '$150.99', 'nm': 'parfum spray (new packaging) 3 oz', 'Design House': 'Burberry', 'Fragrance Notes': 'Jasmine, Rose, Peach, Patchouli, Amber', 'Year': '2016'},
    
    {'index': 'molecule 01', 'Price': '$114.99', 'nm': 'eau de toilette spray 3.5 oz', 'Design House': 'Escentric Molecules', 'Fragrance Notes': 'Iso E Super', 'Year': '2006'}
]

In [1606]:
perfumes_data = perfumes_data.drop('name_char', axis = 1)

In [1710]:
perfumes_data[perfumes_data.duplicated(subset = ['index', 'Price', 'nm', 'Design House'])]

,index,Price,nm,Design House,Fragrance Notes,Year
3218,molecule 01,$114.99,eau de toilette spray 3.5 oz,Escentric Molecules,Iso E Super,2006


In [1649]:
perfumes_data = pd.concat([perfumes_data, pd.DataFrame(missings)], ignore_index=True)

In [1714]:
perfumes_data = perfumes_data.drop_duplicates(subset = ['index', 'Price', 'nm'])

In [1715]:
perfumes_data.to_csv('perfumes_data_almost_clean.csv', index = False)